<a href="https://colab.research.google.com/github/HazemmoAlsady/AWN_Graduation_Project/blob/main/Hazem's%20edits/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd
import joblib
import os

BASE_PATH = "/content/drive/MyDrive/graduation_project"

# Need Level Encoder (الصح)
need_encoder = joblib.load(
    os.path.join(BASE_PATH, "need_level_target_encoder.joblib")
)

# Assistance Type Encoder (الصح)
assist_encoder = joblib.load(
    os.path.join(BASE_PATH, "assistance_type_target_encoder.joblib")
)

print("Need Level Encoder Classes:", need_encoder.classes_)
print("Assistance Type Encoder Classes:", assist_encoder.classes_)


Need Level Encoder Classes: ['disability_equipment' 'education' 'financial' 'food' 'housing' 'medical']
Assistance Type Encoder Classes: ['disability_equipment' 'education' 'financial' 'food' 'housing' 'medical']


In [38]:
print("Need Level Classes:", need_encoder.classes_)


Need Level Classes: ['disability_equipment' 'education' 'financial' 'food' 'housing' 'medical']


In [39]:
orgs = pd.DataFrame([
    {
        "org": "جمعية الأورمان",
        "assistance_types": ["food", "financial", "housing"],
        "cities": ["القاهرة", "الجيزة"],
        "priority": 5
    },
    {
        "org": "جمعية رسالة",
        "assistance_types": ["education", "medical"],
        "cities": ["القاهرة", "الإسكندرية"],
        "priority": 4
    },
    {
        "org": "بنك الطعام",
        "assistance_types": ["food"],
        "cities": ["القاهرة", "الجيزة"],
        "priority": 5
    },
    {
        "org": "جمعية التكافل الاجتماعي",
        "assistance_types": ["financial", "housing"],
        "cities": ["سوهاج", "أسيوط"],
        "priority": 3
    },
    {
        "org": "مؤسسة مجدي يعقوب",
        "assistance_types": ["medical"],
        "cities": ["أسوان"],
        "priority": 5
    },
    {
        "org": "مؤسسة مصر الخير",
        "assistance_types": ["food", "education", "financial", "housing"],
        "cities": ["القاهرة", "الجيزة"],
        "priority": 5
    }
])


In [40]:
def recommend_organizations(
    assistance_type,
    city,
    need_level,
    orgs_df,
    top_n=3
):
    scores = []

    for _, row in orgs_df.iterrows():
        score = 0

        # Assistance match
        if assistance_type in row["assistance_types"]:
            score += 4

        # City match
        if city in row["cities"]:
            score += 3

        # Need level weight
        if need_level == "High":
            score += 2
        elif need_level == "Medium":
            score += 1

        # Org priority
        score += row["priority"]

        scores.append(score)

    ranked = orgs_df.copy()
    ranked["score"] = scores

    return ranked.sort_values("score", ascending=False).head(top_n)


In [41]:
def choose_final_organization(recommendations_df):
    return (
        recommendations_df
        .sort_values("score", ascending=False)
        .iloc[0]["org"]
    )


In [42]:
def full_ai_pipeline(input_df):
    # Need Level
    need_pred = need_model.predict(input_df)[0]
    need_level = need_encoder.inverse_transform([need_pred])[0]

    # Assistance Type
    assist_pred = assist_model.predict(input_df)[0]
    assistance_type = assist_encoder.inverse_transform([assist_pred])[0]

    # Recommendation
    ranked_orgs = recommend_organizations(
        assistance_type=assistance_type,
        city=input_df.iloc[0]["city"],
        need_level=need_level,
        orgs_df=orgs,
        top_n=3
    )

    final_org = choose_final_organization(ranked_orgs)

    return {
        "predicted_need_level": need_level,
        "predicted_assistance_type": assistance_type,
        "final_recommended_org": final_org,
        "top_organizations": ranked_orgs[["org", "score"]]
    }


In [43]:
sample_case = pd.DataFrame([{
    "family_size": 5,
    "income_monthly": 1500,
    "monthly_expenses": 3500,
    "debts": 9000,
    "number_of_children": 3,
    "age": 40,
    "expense_to_income_ratio": 2.3,
    "case_type": "أرملة",
    "housing_type": "إيجار",
    "health_status": "مزمن",
    "city": "القاهرة",
    "gender": "female",
    "request_text_clean": "تحتاج مساعدة عاجلة في السكن"
}])

result = full_ai_pipeline(sample_case)
result


{'predicted_need_level': 'education',
 'predicted_assistance_type': 'education',
 'final_recommended_org': 'مؤسسة مصر الخير',
 'top_organizations':                org  score
 5  مؤسسة مصر الخير     12
 1      جمعية رسالة     11
 0   جمعية الأورمان      8}